<a href="https://colab.research.google.com/github/KieranDingwall/Natural-Language-Processing-Coursework---Kieran-Dingwall/blob/main/NLP_Coursework_V2_(Colab_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Coursework

By Kieran Dingwall - Student Number: 2208619

### Section 1 - Dataset Exploring and Pre Processing

In [1]:
# Imports
import pandas as pd
import numpy as np
# import io

# Load Dataset
from google.colab import files, drive #import colab functionality
drive.mount("/content/drive")
path = '/content/drive/MyDrive/Uni/4th Year 1st Sem/Natural Language Processing/NLP Coursework/pricerunner_aggregate.csv' # May need to change path
product_data = pd.read_csv(path)
print(product_data)

Mounted at /content/drive
       Product ID                                      Product Title  \
0               1                    apple iphone 8 plus 64gb silver   
1               2                apple iphone 8 plus 64 gb spacegrau   
2               3  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
3               4                apple iphone 8 plus 64gb space grey   
4               5  apple iphone 8 plus gold 5.5 64gb 4g unlocked ...   
...           ...                                                ...   
35306       47350  smeg fab28 60cm retro style right hand hinge f...   
35307       47351  smeg fab28 60cm retro style left hand hinge fr...   
35308       47352  smeg fab28 60cm retro style left hand hinge fr...   
35309       47355     candy 60cm built under larder fridge cru160nek   
35310       47358           neff k4316x7gb built under larder fridge   

        Merchant ID   Cluster ID             Cluster Label   Category ID  \
0                 1            1 

In [2]:
# Remove unused columns from data
product_df = product_data.drop(product_data.columns[[0, 2, 3, 5]], axis=1)
print(product_df)

                                           Product Title  \
0                        apple iphone 8 plus 64gb silver   
1                    apple iphone 8 plus 64 gb spacegrau   
2      apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
3                    apple iphone 8 plus 64gb space grey   
4      apple iphone 8 plus gold 5.5 64gb 4g unlocked ...   
...                                                  ...   
35306  smeg fab28 60cm retro style right hand hinge f...   
35307  smeg fab28 60cm retro style left hand hinge fr...   
35308  smeg fab28 60cm retro style left hand hinge fr...   
35309     candy 60cm built under larder fridge cru160nek   
35310           neff k4316x7gb built under larder fridge   

                  Cluster Label  Category Label  
0      Apple iPhone 8 Plus 64GB   Mobile Phones  
1      Apple iPhone 8 Plus 64GB   Mobile Phones  
2      Apple iPhone 8 Plus 64GB   Mobile Phones  
3      Apple iPhone 8 Plus 64GB   Mobile Phones  
4      Apple iPhone 8 Plus 64

In [3]:
# Convert the feature columns into numpy array's
x = product_df["Product Title"].to_numpy()
y = product_df[" Category Label"].to_numpy()

# Testing outputs
#print(x)
#print(y)

In [4]:
# Checking Balance

print(product_df["Product Title"].value_counts())
print(product_df[" Category Label"].value_counts())

# Balance is good

Product Title
washing machine                                     90
built in fully integrated dishwasher                35
frost free fridge freezer                           34
washer dryer                                        24
american fridge freezer                             22
                                                    ..
sony hx400v compact camera with 50x optical zoom     1
sony hx400 20mp 50x zoom bridge camera               1
sony cyber shot hx400 digital camera                 1
sony cyber shot dsc hx400v digital camera            1
x t3 body only silver                                1
Name: count, Length: 30993, dtype: int64
 Category Label
Fridge Freezers     5501
Mobile Phones       4081
Washing Machines    4044
CPUs                3862
Fridges             3584
TVs                 3564
Dishwashers         3424
Digital Cameras     2697
Microwaves          2342
Freezers            2212
Name: count, dtype: int64


In [32]:
from sklearn.base import BaseEstimator, TransformerMixin
import nltk #import the natural language toolkit
nltk.download('punkt') #download the package in nltk which supports tokenization
nltk.download('punkt_tab')
nltk.download('stopwords') #download the nltk package for stopwords

from nltk.tokenize import word_tokenize #import the tokenize package
from nltk.corpus import stopwords #import the package from the corpus
from nltk.stem.snowball import SnowballStemmer #import the snowball stemmer (also known as Porter2)


class pre_process(BaseEstimator, TransformerMixin):

    def __init__(self):
      return None #we do not need any parameters to instantiate this class

    def fit(self, X, y=None): #both fit and transform expect the data instances and labels to be called - we do not use the labels, so set y=None
        return self #as explained above, we will not use the fit method

    def transform(self, X, y=None):
      prep_text = []
      for x in X: #for each sentence in the whole dataset
            token_text = word_tokenize(x) #tokenize the document
            normd_text = [token.lower() for token in token_text if token.isalpha()] #list compression to apply some simple cleaning (lower case casting and punctuation removal) to tokenized terms

            swr_text = [token for token in normd_text if token not in stopwords.words('english')] #list compression to remove any stopwords from our list

            stemmer = SnowballStemmer("english") #specify we are using the English stemming rules, as other languages are present in toolkit
            prep_text += [[stemmer.stem(word) for word in swr_text]] #list compression for applying the stemmer

      prep_sentences = [" ".join(sentence) for sentence in prep_text] #we join the sentences back together to ensure compatibility with CountVec, which is doing some of it's own prep
      return prep_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Section 2 - Representation Learning

In [33]:
# Representation Learner - Vectorising
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_base = TfidfVectorizer(max_features=2000).fit_transform(x)
#print(tfidf_base)

### Section 3 - Algorithms

In [34]:
# Testing with K-Nearest Neighbour in a pipeline

# SKLearn Imports
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC

knn_acc_score = []

pipeline = Pipeline([
  ('prep', pre_process()),
  ('vect', CountVectorizer()),
  ('tfidf_base', TfidfTransformer()),
  ('clf', KNeighborsClassifier(n_neighbors=5))
])

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]

  pipeline.fit(x_train, y_train)
  predictions = pipeline.predict(x_test)
  acc = accuracy_score(predictions, y_test)
  knn_acc_score.append(acc)

print("Accuracy:", np.mean(knn_acc_score))

Accuracy: 0.8531044846893796


In [35]:
# Testing with Multi Layer Perception to compare accuracy scores
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# SKLearn Imports
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

def mlp(input_dimension, num_classes):
  model = Sequential()
  model.add(Dense(128, input_shape=(input_dimension,), activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  return model

kf = StratifiedKFold(n_splits=5, shuffle=True)
# Initialize acc_score list
mlp_acc_score = []

# Map string labels to integers for to_categorical and StratifiedKFold
unique_labels = np.unique(y)
label_to_int = {label: i for i, label in enumerate(unique_labels)}
y_encoded = np.array([label_to_int[label] for label in y])

NUM_CLASSES = len(unique_labels)
TFIDF_MAX_FEATURES = 2000

for train_index, test_index in kf.split(x, y_encoded):
  x_train, x_test = x[train_index], x[test_index]
  y_train_encoded, y_test_encoded = y_encoded[train_index], y_encoded[test_index]

  # Initialize a new TfidfVectorizer for each fold to prevent data leakage
  tfidf_vectorizer = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES)

  x_train_transformed = tfidf_vectorizer.fit_transform(x_train)
  x_train_dense = x_train_transformed.todense() # Convert to dense matrix for Keras

  x_test_transformed = tfidf_vectorizer.transform(x_test)
  x_test_dense = x_test_transformed.todense() # Convert to dense matrix for Keras

  # Instantiate a new mlp model for each fold and ensure correct input/output dimensions
  model = mlp(TFIDF_MAX_FEATURES, NUM_CLASSES)
  y_train_one_hot = to_categorical(y_train_encoded, NUM_CLASSES) # Convert y to one hot vectors
  y_test_one_hot = to_categorical(y_test_encoded, NUM_CLASSES)

  # Configure the model and start training (using categorical_crossentropy for multi-class)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x_train_dense, y_train_one_hot, epochs=10, batch_size=250, verbose=1, validation_split=0.2)

  # Test the model after training
  test_results = model.evaluate(x_test_dense, y_test_one_hot, verbose=1)
  print(f'Test results - Loss: {test_results[0]:.4f} - Accuracy: {test_results[1]*100:.2f}%')

  mlp_acc_score.append(test_results[1])

print("Accuracy:", np.mean(mlp_acc_score))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5737 - loss: 1.7506 - val_accuracy: 0.3628 - val_loss: 3.8784
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9673 - loss: 0.1273 - val_accuracy: 0.3915 - val_loss: 4.6818
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9770 - loss: 0.0746 - val_accuracy: 0.4000 - val_loss: 4.9781
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9795 - loss: 0.0593 - val_accuracy: 0.4046 - val_loss: 5.2249
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9756 - loss: 0.0596 - val_accuracy: 0.3931 - val_loss: 5.3658
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9796 - loss: 0.0530 - val_accuracy: 0.4179 - val_loss: 5.4933
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9771 - loss: 0.0545 - val_accuracy: 0.4034 - val_loss: 5.4936
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9801 - loss: 0.0500 - val_accuracy: 0.4124 - val_l

### Section 4 - Evaluation

Accuracy almost the exact same - expand this